# Self-Distillation mit großem Datensatz

In dieser Version habe ich nicht nur einen großen Datensatz, sondern auch ein großes fehlerfreies Evaluationsset und Farbe und Art der Figuren.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

2023-02-27 12:07:18.047763: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-27 12:07:18.882172: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/cuda/lib64:/usr/local/cuda-11.2/lib64
2023-02-27 12:07:18.882212: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/cuda/lib64:/usr/local/cuda-11.2/lib64
2023-02-27 12:07:18.882216: W te

'2.11.0'

### Load data

In [2]:
label2array = {
     'WB' : np.array([1,0,0,0,0,0,0,0,0,0,0,0,0]), 
     'E'  : np.array([0,1,0,0,0,0,0,0,0,0,0,0,0]), 
     'WK' : np.array([0,0,1,0,0,0,0,0,0,0,0,0,0]), 
     'WN' : np.array([0,0,0,1,0,0,0,0,0,0,0,0,0]), 
     'WP' : np.array([0,0,0,0,1,0,0,0,0,0,0,0,0]), 
     'WQ' : np.array([0,0,0,0,0,1,0,0,0,0,0,0,0]), 
     'WR' : np.array([0,0,0,0,0,0,1,0,0,0,0,0,0]),
     'BB' : np.array([0,0,0,0,0,0,0,1,0,0,0,0,0]), 
     'BK' : np.array([0,0,0,0,0,0,0,0,1,0,0,0,0]), 
     'BN' : np.array([0,0,0,0,0,0,0,0,0,1,0,0,0]), 
     'BP' : np.array([0,0,0,0,0,0,0,0,0,0,1,0,0]), 
     'BQ' : np.array([0,0,0,0,0,0,0,0,0,0,0,1,0]), 
     'BR' : np.array([0,0,0,0,0,0,0,0,0,0,0,0,1]) }

label2piece = {
     0:'WB',
     1:'E',
     2:'WK',
     3:'WN',
     4:'WP',
     5:'WQ',
     6:'WR',
     7:'BB',
     8:'BK',
     9:'BN',
    10:'BP',
    11:'BQ',
    12:'BR' }

# Test:
for label in label2array:
    index = label2array[label].argmax()
    if label2piece[index] != label:
        print(label2piece[index] ,label, index)



In [3]:

def load_data(path):
    
    categories = []
    images = []
    labels = []

    for file in os.listdir(path):
        if file.endswith('jpeg'):
            im = cv2.imread(path+"/"+file)
            resized = cv2.resize(im, (64,64), interpolation = cv2.INTER_AREA)
            gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
            reshaped = gray.reshape((64,64,1))

            color = file.split('_')[2].upper()
            piece = file.split('_')[3][:1].upper()

            if piece != 'E': 
                category = color + piece
            else:
                category = 'E'
                
            label = label2array[category]
            image = reshaped

            images.append(image)
            labels.append(label)

    images = np.array(images)
    labels = np.array(labels)
    
    return (images,labels)

In [4]:
path = "square_images_color_piece"

train_images, train_labels = load_data(path)

print(len(train_images),train_images[0].shape,train_labels[0].shape)

1190208 (64, 64, 1) (13,)


In [4]:
path = "square_images_color_piece_evaluation"

test_images, test_labels = load_data(path)

print(len(test_images),test_images[0].shape,test_labels[0].shape)

38080 (64, 64, 1) (13,)


### Code

In [9]:
test_images[0,:,:,:].shape
type(test_images[0,:,:,:])
type(test_images[0,0,0,0])
test_images.shape

(38080, 64, 64, 1)

In [17]:

def make_model():
    opt = tf.keras.optimizers.Adam(
                  learning_rate = 0.001
                )

    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(13, activation='softmax'))

    model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

In [7]:
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [8]:
def train_model(model, train_images, train_labels, eval_data, eval_labels, epochs=1, batch_size=32):
    train_generator = datagen.flow(train_images, train_labels, batch_size=batch_size)
    model.fit(train_generator, 
              steps_per_epoch=len(train_images) // batch_size, 
              validation_data = (eval_data,eval_labels), 
              epochs=epochs
             )
    return model
    

In [9]:
def relabel(model,images):
    output = model.predict(images)
    num_of_samples = output.shape[0]
    new_labels = np.zeros((num_of_samples,7))
    new_label_indices = np.argmax(output,axis=1)
    for i,index in enumerate(new_label_indices):
        new_labels[i,index] = 1.0
    return new_labels

def evaluate(model,train_images,train_labels):
    results = model.evaluate(train_images,train_labels)
    return np.array(results)

def sanity_check(train_labels,train_labels_x):
    num_labels = train_labels.shape[0]
    count = 0
    for t in range(num_labels):
        if (train_labels[t] == train_labels_x[t]).all():
            count += 1
    print('Sanity check:',count/num_labels)
    return count/num_labels

### Training

In [40]:
model = make_model()

train_model(model, train_images, train_labels, test_images, test_labels, epochs=1, batch_size=32)

2023-02-23 05:56:11.504399: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f10700291a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-23 05:56:11.504423: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-02-23 05:56:11.514091: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-23 05:56:11.629064: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


37194/37194 [==============================] - 301s 8ms/step - loss: 0.4998 - accuracy: 0.9004 - val_loss: 0.9527 - val_accuracy: 0.7350


In [10]:
model = make_model()

train_model(model, test_images[:25000], test_labels[:25000], test_images[25000:], test_labels[25000:], epochs=10, batch_size=32)

Epoch 1/10


2023-02-23 17:30:57.773312: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-02-23 17:30:59.198819: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-02-23 17:30:59.208238: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x49c68b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-02-23 17:30:59.208254: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2023-02-23 17:30:59.218332: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-02-23 17:30:59.342577: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the 

781/781 [==============================] - 11s 9ms/step - loss: 0.5272 - accuracy: 0.8405 - val_loss: 0.0485 - val_accuracy: 0.9921
Epoch 2/10
781/781 [==============================] - 7s 8ms/step - loss: 0.1206 - accuracy: 0.9642 - val_loss: 0.0191 - val_accuracy: 0.9961
Epoch 3/10
781/781 [==============================] - 7s 8ms/step - loss: 0.0745 - accuracy: 0.9787 - val_loss: 0.0131 - val_accuracy: 0.9974
Epoch 4/10
781/781 [==============================] - 6s 8ms/step - loss: 0.0473 - accuracy: 0.9870 - val_loss: 0.0122 - val_accuracy: 0.9976
Epoch 5/10
781/781 [==============================] - 7s 8ms/step - loss: 0.0456 - accuracy: 0.9874 - val_loss: 0.0117 - val_accuracy: 0.9979
Epoch 6/10
781/781 [==============================] - 8s 10ms/step - loss: 0.0317 - accuracy: 0.9913 - val_loss: 0.0169 - val_accuracy: 0.9958
Epoch 7/10
781/781 [==============================] - 6s 8ms/step - loss: 0.0305 - accuracy: 0.9928 - val_loss: 0.0119 - val_accuracy: 0.9981
Epoch 8/10
781/

In [54]:
model.evaluate(train_images,train_labels)

37194/37194 [==============================] - 48s 1ms/step - loss: 5.5251 - accuracy: 0.9474


[5.525136947631836, 0.9474411010742188]

In [11]:
# Ein Plan: model auf test-daten trainiert auf train_data anwenden:
# Die Samples rauschmeißen wo vorhersage und label sich widersprechen
# Dann auf dem Datensatz trainieren ...

new_labels = model.predict(train_images)

new_labels.shape

37194/37194 [==============================] - 32s 868us/step


(1190208, 13)

In [12]:
new_labels[0,:]

array([1.75910161e-16, 3.72638084e-14, 6.55198823e-14, 1.19914675e-14,
       5.77327411e-13, 7.72137636e-19, 2.17027163e-15, 2.09013024e-07,
       2.55748574e-13, 7.43772048e-08, 9.99999642e-01, 4.84452659e-16,
       5.19559007e-10], dtype=float32)

In [13]:
train_labels.astype('float32')[0,:]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

In [14]:
new_train_images = []
new_train_labels = []
for t in range(new_labels.shape[0]):
    if train_labels[t,:].argmax() == new_labels[t,:].argmax():
        new_train_images.append(train_images[t,:])
        new_train_labels.append(train_labels[t,:])

new_train_images = np.array(new_train_images)
new_train_labels = np.array(new_train_labels)
        
new_train_images.shape

(1126843, 64, 64, 1)

In [21]:
#model = make_model()

train_model(model, new_train_images, new_train_labels, test_images, test_labels, epochs=5, batch_size=64)


Epoch 1/5
17606/17606 [==============================] - 267s 15ms/step - loss: 0.0202 - accuracy: 0.9963 - val_loss: 0.0042 - val_accuracy: 0.9993
Epoch 2/5
17606/17606 [==============================] - 270s 15ms/step - loss: 0.0275 - accuracy: 0.9953 - val_loss: 0.0068 - val_accuracy: 0.9991
Epoch 3/5
17606/17606 [==============================] - 269s 15ms/step - loss: 0.0375 - accuracy: 0.9934 - val_loss: 0.0074 - val_accuracy: 0.9987
Epoch 4/5
17606/17606 [==============================] - 268s 15ms/step - loss: 0.0414 - accuracy: 0.9926 - val_loss: 0.0124 - val_accuracy: 0.9992
Epoch 5/5
17606/17606 [==============================] - 266s 15ms/step - loss: 0.0476 - accuracy: 0.9921 - val_loss: 0.0049 - val_accuracy: 0.9989


In [20]:
model.save('piece_color_classification_09994')

INFO:tensorflow:Assets written to: piece_color_classification_09994/assets


INFO:tensorflow:Assets written to: piece_color_classification_09994/assets
